In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import threading

spark = SparkSession.builder \
    .appName("Parallel Jobs Execution") \
    .master("local[*]") \
    .getOrCreate()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 21:03:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def run_job(args):
    rdd = spark.sparkContext.parallelize(range(0,100000000))
    rdd = rdd.map(lambda x: {"id":x * args})
    rdd.toDF().select(f.max("id")).show()
    # .write.csv("data_"+str(args),mode="overwrite")



In [3]:
t_l = {} 
for i in range(10):
    x = threading.Thread(target=run_job,args=(i,))
    t_l[i]= x
    print(i,"--->starting")
    x.start()
for j in t_l:
    print("waiting to complete ", j)
    t_l[j].join()



0 --->starting
1 --->starting
2 --->starting
3 --->starting
4 --->starting
5 --->starting
6 --->starting
7 --->starting
8 --->starting
9 --->starting
waiting to complete  0


24/02/23 21:04:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+---------+
|  max(id)|
+---------+
|399999996|
+---------+
+---------+
|  max(id)|
+---------+
|199999998|
+---------+

+---------+
|  max(id)|
+---------+
|799999992|
+---------+

+-------+
|max(id)|
+-------+
|      0|
+-------+

+---------+
|  max(id)|
+---------+
|599999994|
+---------+


+---------+
|  max(id)|
+---------+
|699999993|
+---------+

+---------+
|  max(id)|
+---------+
|499999995|
+---------+

+--------+
| max(id)|
+--------+
|99999999|
+--------+

waiting to complete  1
waiting to complete  2
waiting to complete  3
+---------+
|  max(id)|
+---------+
|899999991|
+---------+



+---------+
|  max(id)|
+---------+
|299999997|
+---------+

waiting to complete  4
waiting to complete  5
waiting to complete  6
waiting to complete  7
waiting to complete  8
waiting to complete  9
